# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены.
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания:
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [17]:
import re
from nltk.metrics import edit_distance

def calculate_wer(reference_text: str, recognized_text: str) -> float:
  #приведение к нижнему регистру, удаление пунктуации и разбивка на слова
  reference_words = re.sub(r'[^\w\s]', '', reference_text.lower()).split()
  recognized_words = re.sub(r'[^\w\s]', '', recognized_text.lower()).split()

  # расстояние Левенштейна
  distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

  for i in range(len(reference_words) + 1):
    distance_matrix[i][0] = i

  for j in range(len(recognized_words) + 1):
    distance_matrix[0][j] = j

  for i in range(1, len(reference_words) + 1):
    for j in range(1, len(recognized_words) + 1):
      insertion = distance_matrix[i][j-1]+1
      deletion = distance_matrix[i-1][j]+1
      substitution = distance_matrix[i-1][j-1] + (reference_words[i-1] != recognized_words[j-1])
      distance_matrix[i][j] = min(insertion, deletion, substitution)

  # Расчет WER (в процентах)
  wer = (distance_matrix[-1][-1] / len(reference_words)) * 100.0
  return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [18]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"

def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")

test_wer()


Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания.

Значение WER получается из трех видов ошибок:
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется.

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга.

пример выравнивания:

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [19]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [20]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
  # Перенесите сюда код из задания 1.a.
  reference_words = re.sub(r'[^\w\s]', '', reference_text.lower()).split()
  recognized_words = re.sub(r'[^\w\s]', '', recognized_text.lower()).split()
  distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
  for i in range(len(reference_words) + 1):
    distance_matrix[i][0] = i
  for j in range(len(recognized_words) + 1):
    distance_matrix[0][j] = j
  for i in range(1, len(reference_words) + 1):
    for j in range(1, len(recognized_words) + 1):
      insertion = distance_matrix[i][j - 1] + 1
      deletion = distance_matrix[i - 1][j] + 1
      substitution = distance_matrix[i - 1][j - 1] + (reference_words[i - 1] != recognized_words[j - 1])
      distance_matrix[i][j] = min(insertion, deletion, substitution)

  wer = (distance_matrix[-1][-1] / len(reference_words)) * 100.0

  # используя distance_matrix восстановите путь (набор операций), который соответствует найденому WER
    #TODO

  # Выравнивание слов
  i, j = len(reference_words), len(recognized_words)
  ali_ref, ali_rec, ali_ann = [], [], []
  correct, deletion, insertion, substitution = 0, 0, 0, 0

  while i > 0 or j > 0:
    current_distance = distance_matrix[i][j]

    if i > 0 and distance_matrix[i - 1][j] == current_distance - 1:
      # Удаление
      ali_ref.append(reference_words[i - 1])
      ali_rec.append("***")
      ali_ann.append("D")
      deletion += 1
      i -= 1
    elif j > 0 and distance_matrix[i][j - 1] == current_distance - 1:
      # Вставка
      ali_ref.append("***")
      ali_rec.append(recognized_words[j - 1])
      ali_ann.append("I")
      insertion += 1
      j -= 1
    else:
      # Замена или совпадение
      ali_ref.append(reference_words[i - 1])
      ali_rec.append(recognized_words[j - 1])
      if reference_words[i - 1] != recognized_words[j - 1]:
        ali_ann.append("S")
        substitution += 1
      else:
        ali_ann.append("C")
        correct += 1
      i -= 1
      j -= 1

  # Обратный порядок выравнивания
  ali_ref.reverse()
  ali_rec.reverse()
  ali_ann.reverse()

  ali = [ali_ref, ali_rec, ali_ann]

  # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
  # ali[1] = разбитая по словам гипотеза.
  # ali[2] = аннотация

  assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"

  # Возврат результатов
  return {
    "wer": wer,
    "cor": correct,
    "del": deletion,
    "ins": insertion,
    "sub": substitution,
    "ali": ali
  }

In [ ]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    return {
            "wer" : 0,
            "cor": 2,
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]}

In [21]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"


def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2,
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1,
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1,
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1,
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")

test_wer_with_alignment()

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания.

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание:
Напишите функцию, которая кроме стандартного WER считает дополнительно RichTranscriptErrorRate (RTER) по формуле

$$ RTER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [22]:
import nltk
nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
def calculate_wer_per(reference_text: str, recognized_text: str):
  # TODO
  # Токенизация слов и знаков препинания
  reference_words = wordpunct_tokenize(reference_text)
  recognized_words = wordpunct_tokenize(recognized_text)

  # Матрица расстояний Левенштейна
  distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
  for i in range(len(reference_words) + 1):
    distance_matrix[i][0] = i
  for j in range(len(recognized_words) + 1):
    distance_matrix[0][j] = j

  for i in range(1, len(reference_words) + 1):
    for j in range(1, len(recognized_words) + 1):
      insertion = distance_matrix[i][j - 1] + 1
      deletion = distance_matrix[i - 1][j] + 1
      substitution = distance_matrix[i - 1][j - 1] + (reference_words[i - 1] != recognized_words[j - 1])

      # Бесконечное расстояние для замены слов и знаков препинания
      if (reference_words[i - 1].isalpha() and not recognized_words[j - 1].isalpha()) or \
        (not reference_words[i - 1].isalpha() and recognized_words[j - 1].isalpha()):
        substitution = float('inf')

      distance_matrix[i][j] = min(insertion, deletion, substitution)

  # Выравнивание слов
  i, j = len(reference_words), len(recognized_words)
  ali_ref, ali_rec, ali_ann = [], [], []
  correct, deletion, insertion, substitution = 0, 0, 0, 0
  correct_p, deletion_p, insertion_p, substitution_p = 0, 0, 0, 0

  while i > 0 or j > 0:
    current_distance = distance_matrix[i][j]

    if i > 0 and distance_matrix[i - 1][j] == current_distance - 1:
      # Удаление
      ali_ref.append(reference_words[i - 1])
      ali_rec.append("***")
      if not reference_words[i - 1].isalpha():
        ali_ann.append("D_p")
        deletion_p += 1
      else:
        ali_ann.append("D")
        deletion += 1
      i -= 1
    elif j > 0 and distance_matrix[i][j - 1] == current_distance - 1:
      # Вставка
      ali_ref.append("***")
      ali_rec.append(recognized_words[j - 1])
      if not recognized_words[j - 1].isalpha():
        ali_ann.append("I_p")
        insertion_p += 1
      else:
        ali_ann.append("I")
        insertion += 1
      j -= 1
    else:
      # Замена
      ali_ref.append(reference_words[i - 1])
      ali_rec.append(recognized_words[j - 1])
      if reference_words[i - 1] != recognized_words[j - 1]:
        if not reference_words[i - 1].isalpha() and not recognized_words[j - 1].isalpha():
          ali_ann.append("S_p")
          substitution_p += 1
        else:
          ali_ann.append("S")
          substitution += 1
      else:
        if not reference_words[i - 1].isalpha() and not recognized_words[j - 1].isalpha():
          ali_ann.append("C_p")
          correct_p += 1
        else:
          ali_ann.append("C")
          correct += 1
      i -= 1
      j -= 1

  # Обратный порядок выравнивания
  ali_ref.reverse()
  ali_rec.reverse()
  ali_ann.reverse()

  ali = [ali_ref, ali_rec, ali_ann]

  # Расчет WER и PER
  wer = (insertion + deletion + substitution) * 100 / (deletion + substitution + correct)
  per = (insertion_p + deletion_p + substitution_p) * 100 / (deletion_p + substitution_p + correct_p)

  # Возврат результатов
  return {"wer": wer, "per": per, "ali": ali}

In [24]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"


def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")

test_wer_per()

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация.

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки.



In [25]:
import itertools

In [32]:
def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list) and isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)

    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов
    # и выберете тот, который соответствует минимальному SA-WER

    # Инициализация матрицы расстояний
    distance_matrix = [[i if j == 0 else j for j in range(len(recognized_text) + 1)] for i in range(len(reference_text) + 1)]

    for i in range(1, len(reference_text) + 1):
        for j in range(1, len(recognized_text) + 1):
            insertion = distance_matrix[i][j - 1] + 1
            deletion = distance_matrix[i - 1][j] + 1
            substitution = distance_matrix[i - 1][j - 1] + (reference_text[i - 1] != recognized_text[j - 1])
            distance_matrix[i][j] = min(insertion, deletion, substitution)

    # используя distance_matrix восстановите путь (набор операций), который соответствует найденому WER
    #TODO
    speaker_names = list(set(reference_speakers))
    speaker_ids = list(set(recognized_speakers))

    speaker_ids += [None] * (len(speaker_names) - len(speaker_ids))  # Дополнение до равного количества

    min_sawer = None
    ali = []

    for speaker_ids_perm in itertools.permutations(speaker_ids):
        mapping = dict(zip(speaker_names, speaker_ids_perm))
        correct = deletion = insertion = substitution = speaker_incorrect = 0
        ali_ref, ali_rec, ali_ann = [], [], []

        i, j = len(reference_text), len(recognized_text)

        while i > 0 or j > 0:
            current_distance = distance_matrix[i][j]
            # Проверяем, является ли текущий элемент удалением
            if i > 0 and distance_matrix[i - 1][j] == current_distance - 1:
                ali_ref.append(reference_text[i - 1])
                ali_rec.append("***")
                ali_ann.append("D")
                deletion += 1
                i -= 1
            # Проверяем, является ли текущий элемент вставкой
            elif j > 0 and distance_matrix[i][j - 1] == current_distance - 1:
                ali_ref.append("***")
                ali_rec.append(recognized_text[j - 1])
                ali_ann.append("I")
                insertion += 1
                j -= 1
            # Случай, когда элементы совпадают или различаются
            else:
                ali_ref.append(reference_text[i - 1])
                ali_rec.append(recognized_text[j - 1])
                if reference_text[i - 1] != recognized_text[j - 1]:
                    ali_ann.append("S")
                    substitution += 1
                elif mapping[reference_speakers[i - 1]] != recognized_speakers[j - 1]:
                    ali_ann.append("S_I")
                    speaker_incorrect += 1
                else:
                    ali_ann.append("C")
                    correct += 1
                i -= 1
                j -= 1

        # Вычисление SAWER (Speech Accuracy with Error Rate)
        sawer = 100 * (insertion + deletion + substitution + speaker_incorrect) / (deletion + substitution + correct + speaker_incorrect)
        # Обновляем минимальное значение SAWER и сохраняем выравнивание, если текущее значение меньше предыдущего
        if min_sawer is None or sawer < min_sawer:
            min_sawer = sawer
            ali = [ali_ref[::-1], ali_rec[::-1], ali_ann[::-1]]  # Обратный порядок

    return {"sawer": min_sawer, "ali": ali}


In [33]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]


def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")

test_sawer()

Test 3 passed
